In [143]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
PATH = "/home/kasia/code/chromedriver"





In [369]:
def scrap_product(article_code, item_count):
    driver = webdriver.Chrome(executable_path=PATH)
    driver.get(f"https://www.ikea.com/de/de/search/products/?q=s{article_code}")
    
    X = driver.find_element_by_class_name('results__list')
    tag = X.find_element_by_tag_name('a')
    #go to product page
    tag.click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    X = soup.find_all("div",  {"id": 'SEC_product-details-packaging'})
    return X[0]

article_code = '39128564'
page = scrap_product(article_code, 2)




In [370]:
page



<div aria-labelledby="product-details-packaging" class="range-revamp-accordion__content" id="SEC_product-details-packaging" role="region"><div><span class="range-revamp-product-details__package-count">Dieses Produkt besteht aus 16 Paketen</span><div class="range-revamp-product-details__container"><span class="range-revamp-product-details__header notranslate">KOMPLEMENT</span><span>Kleiderstange</span><span class="range-revamp-product-details__label">Artikelnummer:</span><span class="range-revamp-product-identifier" x-ms-format-detection="none"><span class="range-revamp-product-identifier__value">902.568.93</span></span><div class="range-revamp-product-details__container"><span class="range-revamp-product-details__label">Breite: 6 cm</span><span class="range-revamp-product-details__label">Höhe: 4 cm</span><span class="range-revamp-product-details__label">Länge: 49 cm</span><span class="range-revamp-product-details__label">Gewicht: 0.28 kg</span><span class="range-revamp-product-details_

In [371]:
def extract_numeric_product_info(y):
    """
    ['Artikelnummer:',
     'Breite: 61 cm',
     'Höhe: 7 cm',
     'Länge: 245 cm',
     'Gewicht: 44.16 kg',
     'Paket(e): 2']
    """
    numeric_product_info = []
    for x in [item for subitem in y for item in subitem.split()]:   
        try:
            x = float(x) 
            numeric_product_info.append(x)       
        except (ValueError, TypeError):
            pass
    return numeric_product_info

def packages_dimensions_weights(page):
    """
    [(
    article_code (str),
    item_count (int),
    [(
        package_id (int),
        package_length (int),
        package_width (int),
        package_height (int),
        package_weight (float)
    )])]
    """
    info = page.find_all('div',  {"class": 'range-revamp-product-details__container'})
    number = page.find_all('span',  {"class": 'range-revamp-product-identifier__value'})
    list_info = []
    for i,(x,y) in enumerate(zip(number,info),start = 1):


        y = [info.text for info in y.find_all('span',  {"class": 'range-revamp-product-details__label'})]

        list_info.append([(x.text,i),extract_numeric_product_info(y)])  
    return list_info
    
packages_dimensions_weights(page)



[[('902.568.93', 1), [6.0, 4.0, 49.0, 0.28, 1.0]],
 [('402.568.95', 2), [6.0, 4.0, 49.0, 0.28, 1.0]],
 [('302.779.59', 3), [6.0, 4.0, 74.0, 0.43, 1.0]],
 [('802.145.68', 4), [6.0, 4.0, 74.0, 0.43, 1.0]],
 [('202.145.71', 5), [48.0, 2.0, 59.0, 3.35, 6.0]],
 [('102.463.32', 6), [48.0, 2.0, 59.0, 3.35, 6.0]],
 [('902.779.61', 7), [61.0, 7.0, 245.0, 44.16, 2.0]]]

In [386]:
info = page.find_all('div',  {"class": 'range-revamp-product-details__container'})
number = page.find_all('span',  {"class": 'range-revamp-product-identifier__value'})





subproduct_info = {}
whole_product = {}
for i,(x,y) in enumerate(zip(number,info)):
    y = [info.text for info in y.find_all('span',  {"class": 'range-revamp-product-details__label'})]

    subproduct_info[x.text] = extract_numeric_product_info(y)
    

whole_product[article_code] = subproduct_info

pd.DataFrame.from_records(whole_product)



,39128564
102.463.32,"[48.0, 2.0, 59.0, 3.35, 6.0]"
202.145.71,"[48.0, 2.0, 59.0, 3.35, 6.0]"
302.779.59,"[6.0, 4.0, 74.0, 0.43, 1.0]"
402.568.95,"[6.0, 4.0, 49.0, 0.28, 1.0]"
802.145.68,"[6.0, 4.0, 74.0, 0.43, 1.0]"
902.568.93,"[6.0, 4.0, 49.0, 0.28, 1.0]"
902.779.61,"[61.0, 7.0, 245.0, 44.16, 2.0]"
